# Exploring Suburbs in Greater Sydney

## Introduction 

Sydney is the capital city of New South Wales (NSW) and home to over 60% percent of the state's popolation. The city is made pf 658 suburbs and spreads over 33 local government areas. Sydney is ranked in the top ten most livable cities but also one the the most expensive cities in the world. It's the financial capital of Australia, and some of Australian top universities are also located in Sydeny. Sydney has attracted both domestic and oversea people to start a new life here.

The aim of this project is to explore suburbs in Sydney and find optimal location choices for residents fresh off the boat to help them adapt to the new living environment based on the top vennues in each borough. The target object includes both students and people who want to pursue their career in Sydney.

As a global city open to new immigrants and respecting aboriginal residents, Sydney embraces different ethnicities and cultures; therefore, diiferent communities have their unique vennues.

# Data

Second-level Administrative Divisions, Australia, 2015 (https://geo.nyu.edu/catalog/stanford-fc944xn1421)
The spatial data utilized in this project is imported from New York University's (NYU) Spatial Data Repository. The polygon shapefile contains second-level administrative divisions of Australia. Level 2 divisions include local government areas, including aboriginal councils, boroughs, cities, community government councils, district councils, municipailities, rural cities, shires and territories.

Then Foursquare location data was applied to exlore the top common vennues in each suburb.